In [16]:
import pandas as pd
import numpy as np

In [29]:
df = pd.read_csv("..//src//data_ingestion//stock_data.csv" )

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1006 entries, 0 to 1005
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       1006 non-null   object 
 1   Open       1006 non-null   float64
 2   High       1006 non-null   float64
 3   Low        1006 non-null   float64
 4   Close      1006 non-null   float64
 5   Adj Close  1006 non-null   float64
 6   Volume     1006 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 55.1+ KB


In [23]:
df.iloc[3,4] = np.nan
df.iloc[2,4] = np.nan

In [19]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

In [31]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [32]:
df.head()

,date,open,high,low,close,adj_close,volume
0,2020-01-02,74.059998,75.150002,73.797501,75.087502,73.059425,135480400
1,2020-01-03,74.287498,75.144997,74.125000,74.357498,72.349144,146322800
2,2020-01-06,73.447502,74.989998,73.187500,74.949997,72.925629,118387200
3,2020-01-07,74.959999,75.224998,74.370003,74.597504,72.582657,108872000
4,2020-01-08,74.290001,76.110001,74.290001,75.797501,73.750237,132079200


In [34]:
df['open'] = df['open'].interpolate(method='linear')
df['high'] = df['high'].interpolate(method='linear')
df['low'] = df['low'].interpolate(method='linear')
df['close'] = df['close'].interpolate(method='linear')
df['adj_close'] = df['adj_close'].interpolate(method='linear')
df['volume'] = df['volume'].interpolate(method='linear')